In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('dataset.tsv',sep='\t')

In [3]:
data

,Id_UT,Toponimo,Comune,Tav_IGM,Masl,Tipologia_UT,Tipologia_Sito,Descrizione,X,Y,Compilatore,Responsabile,m x m,Ha,Latitudine,Longitudine,Anomalia
0,R1,Contrada Palma,Ramacca,F. 269 III NE,82,Area di frammenti,n.d.,Contrada Palma è situata immediatamente a N de...,473476.00,4141868.00,Laureanti 2007,NaN,5 x 5,0.0025,37.423244,14.700228,GOOGLE MAPS
1,R100,Poggio Callura,Palagonia,F. 269 II SO,58,Area di frammenti,n.d.,Poggio Callura è una bassa collina ubicata in...,483054.00,4136515.00,Privitera 2003,NaN,250 x 200,0.9000,37.375218,14.808600,NaN
2,R101,Poggio Sciccaria,Palagonia,F. 269 II SO,77,Area di frammenti,n.d.,Poggio Sciccaria è una bassa collina (m 101 m...,482392.00,4136380.00,Privitera 2003,NaN,30 x 30,0.0900,37.374031,14.823631,NaN
3,R102,Contrada Callura I,Palagonia,F. 269 II SO,44,Area di frammenti,n.d.,La Contrada Callura è situata a ca. m 700 a S ...,484390.00,4136379.00,Privitera 2003,NaN,40 x 40,0.1400,37.374016,14.823693,NaN
4,R103,Contrada Callura II,Palagonia,F. 269 II SO,43,Area di frammenti,Fattoria,L'area di frammenti fittili è situata in Contr...,484285.00,4136634.00,Privitera 2003,NaN,100 x 80,0.8000,37.376318,14.822508,NaN
5,R104,Contrada Callura III,Palagonia,F. 269 II SO,42,Area di frammenti,n.d.,L'area di frammenti fittili è situata in Contr...,484470.00,4136796.00,Privitera 2003,NaN,20 x 35,0.8500,37.377776,14.824594,NaN
6,R105,Poggio Tecchio,Palagonia,F. 269 II SO,42,Area di frammenti,Frequentazione,Poggio Tecchio è una bassa collina (m 77 slm) ...,484394.00,4135529.00,Privitera 2003,NaN,5 x 5,0.0025,37.366359,14.823757,NaN
7,R106,Poggio Raffo,Palagonia,F. 269 II SO,146,Area di frammenti,n.d.,Poggio Raffo è una collina che raggiunge l'al...,479677.00,4133113.00,Privitera 2003,NaN,NaN,NaN,37.344485,14.770551,NaN
8,R107,Contrada Trefontane,Palagonia,F. 269 II SO,144,Area di frammenti,n.d.,L'area di frammenti fittili (m 30 x 30) è sit...,479939.00,4132199.00,Privitera 2003,NaN,40 x 30,0.0900,37.336261,14.773545,NaN
9,R108,Poggio Fiumefreddo,Lentini,F. 269 II SO,127,Area di frammenti,n.d.,Poggio Fiumefreddo (m 174 slm) sorge nell'area...,482687.00,4131878.00,Privitera 2003,NaN,100 x 30,0.0300,37.333420,14.804564,NaN


In [4]:
data.shape[0]

135

In [5]:
import rdflib
graph = rdflib.Graph()

In [6]:
OntoIRI = rdflib.Namespace('http://www.semanticweb.org/alessiodesktop/ontologies/2019/0/untitled-ontology-10#')

In [7]:
from rdflib.namespace import RDF, OWL, XSD
from math import isnan
from tqdm import tqdm_notebook

In [8]:
for i in tqdm_notebook(range(data.shape[0])):
    line = data.values[i]
    individual = rdflib.URIRef(OntoIRI+line[0])
    graph.add((individual,RDF.type, OWL.NamedIndividual))
    graph.add((individual,OntoIRI.haId_UT,rdflib.Literal(line[0],datatype=XSD.string)))
    graph.add((individual,OntoIRI.haToponimo,rdflib.Literal(line[1],datatype=XSD.string)))
    if not pd.isna(line[2]) :
        municipality = rdflib.URIRef(OntoIRI+line[2].replace(' ',''))
        graph.add((individual,OntoIRI.haComune,municipality))
        graph.add((municipality,OntoIRI.haNomeComune, rdflib.Literal(line[2],datatype=XSD.string)))
    posIGM = rdflib.URIRef(OntoIRI+line[0]+'IGM25000')
    if not pd.isna(line[3]):
        graph.add((individual,OntoIRI.haPosIGM25000, posIGM))
        temp = line[3].split()
        sheet = temp[1]
        quadrant = temp[2]
        orientation = temp[3]
        graph.add((posIGM,OntoIRI.haFoglioIGM25000, rdflib.Literal(sheet,datatype=XSD.integer)))
        graph.add((posIGM,OntoIRI.haQuadranteIGM25000, rdflib.Literal(quadrant,datatype=XSD.string)))
        graph.add((posIGM,OntoIRI.haOrientamentoIGM25000, rdflib.Literal(orientation,datatype=XSD.string)))
    graph.add((individual,OntoIRI.haAltitudineMASL, rdflib.Literal(line[4],datatype=XSD.integer)))
    graph.add((individual,OntoIRI.haTipoUT, rdflib.Literal(line[5],datatype=XSD.string)))
    if not pd.isna(line[6]):
        if(line[6]!='n.d.'): graph.add((individual,OntoIRI.haTipoSito, rdflib.Literal(line[6],datatype=XSD.string)))
    graph.add((individual,OntoIRI.haDescrizione, rdflib.Literal(line[7],datatype=XSD.string)))
    graph.add((individual,OntoIRI.haAscissaFuso33, rdflib.Literal(int(line[8]),datatype=XSD.nonNegativeInteger)))
    graph.add((individual,OntoIRI.haOrdinataFuso33, rdflib.Literal(int(line[9]),datatype=XSD.nonNegativeInteger)))
    if not pd.isna(line[10]): 
        graph.add((individual,OntoIRI.haCompilatore, rdflib.Literal(line[10],datatype=XSD.string)))
    if not pd.isna(line[11]): 
        graph.add((individual,OntoIRI.haResponsabile, rdflib.Literal(line[11],datatype=XSD.string)))
    if not pd.isna(line[12]):
        if (line[12]!='n.d.'):
            mxm = line[12].split();
            graph.add((individual,OntoIRI.haDimensioneA, rdflib.Literal(mxm[0].replace(',','.'),datatype=XSD.decimal)))
            graph.add((individual,OntoIRI.haDimensioneB, rdflib.Literal(mxm[2].replace(',','.'),datatype=XSD.decimal)))
    if not pd.isna(line[13]):
        graph.add((individual,OntoIRI.haDimensioneEttari , rdflib.Literal(line[13],datatype=XSD.decimal)))
    if not pd.isna(line[14]):
        graph.add((individual,OntoIRI.haLatitudine, rdflib.Literal(line[14],datatype=XSD.decimal)))
    if not pd.isna(line[15]):
        graph.add((individual,OntoIRI.haLongitudine, rdflib.Literal(line[15],datatype=XSD.decimal)))
    if not pd.isna(line[16]): 
        graph.add((individual,OntoIRI.haAnomalia, rdflib.Literal(line[16],datatype=XSD.string)))

In [9]:
othergraph = rdflib.Graph()
othergraph.parse('OntologiaNoRecordRDFXML.owl')

graph = graph+othergraph

graph.serialize('OntologiaConRecord.owl')